In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from PIL import Image

2023-07-13 11:26:18.334522: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-13 11:26:18.446270: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-13 11:26:18.447004: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 11:26:19.377607: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
tf.config.list_physical_devices('GPU')

2023-07-13 11:26:48.155073: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-13 11:26:48.236165: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[]

In [3]:
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")

In [4]:
# split train and val data

val_split = .2

val_df = train_df.sample(frac=0.2)
train_df = train_df.drop(val_df.index)

In [5]:
# convert labels to categorical encoding

train_labels = pd.get_dummies(train_df.pop("label")).values
val_labels = pd.get_dummies(val_df.pop("label")).values

In [6]:
# reshape images to 2D pixel grid with 1 channel

def df_to_pixels(df, img_shape):
    pixels = df.apply(lambda x: x.values.reshape(img_shape), axis=1)
    pixels = np.asarray(list(pixels.values)).astype('float32')
    return pixels

In [7]:
img_shape = (28,28,1)

train_pixels = df_to_pixels(train_df, img_shape)
val_pixels = df_to_pixels(val_df, img_shape)
test_pixels = df_to_pixels(test_df, img_shape)

In [8]:
# build tensorflow datasets

batch_size = 128

train_ds = tf.data.Dataset.from_tensor_slices((train_pixels, train_labels)).batch(batch_size)
val_ds = tf.data.Dataset.from_tensor_slices((val_pixels, val_labels)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices(test_pixels).batch(1)

2023-07-13 11:06:06.352723: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-13 11:06:06.353509: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


### Build convolutional neural net

In [13]:
inputs = keras.Input(shape=(28, 28, 1))

x = layers.Rescaling(scale = 1./255)(inputs)

x = layers.Conv2D(
        filters=6,
        kernel_size=(5,5),
        activation='relu',
        padding='same'
    )(x)

x = layers.AveragePooling2D(strides=2,padding='same')(x)

x = layers.Conv2D(
        filters=16,
        kernel_size=(5,5),
        activation='relu',
        padding='same'
    )(x)

x = layers.AveragePooling2D(strides=2,padding='same')(x)

x = layers.Flatten()(x)
x = layers.Dense(120, activation='relu')(x)
x = layers.Dense(84, activation='relu')(x)

outputs = layers.Dense(10, activation='softmax')(x)

In [14]:
model = keras.Model(inputs=inputs, outputs=outputs, name="mnist")

In [15]:
model.compile(
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    optimizer='adam'
)

In [16]:
model.summary()

Model: "mnist"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 rescaling_2 (Rescaling)     (None, 28, 28, 1)         0         
                                                                 
 conv2d_4 (Conv2D)           (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d_4 (Averag  (None, 14, 14, 6)        0         
 ePooling2D)                                                     
                                                                 
 conv2d_5 (Conv2D)           (None, 14, 14, 16)        2416      
                                                                 
 average_pooling2d_5 (Averag  (None, 7, 7, 16)         0         
 ePooling2D)                                                 

In [17]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10)

Epoch 1/10


2023-07-13 11:09:14.141092: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype bool and shape [33600,10]
	 [[{{node Placeholder/_1}}]]


261/263 [============================>.] - ETA: 0s - loss: 0.4745 - accuracy: 0.8540

2023-07-13 11:09:18.711506: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype bool and shape [8400,10]
	 [[{{node Placeholder/_1}}]]


263/263 [==============================] - 5s 16ms/step - loss: 0.4730 - accuracy: 0.8545 - val_loss: 0.1956 - val_accuracy: 0.9398
Epoch 2/10
263/263 [==============================] - 4s 15ms/step - loss: 0.1363 - accuracy: 0.9584 - val_loss: 0.1089 - val_accuracy: 0.9670
Epoch 3/10
263/263 [==============================] - 4s 15ms/step - loss: 0.0908 - accuracy: 0.9719 - val_loss: 0.0873 - val_accuracy: 0.9727
Epoch 4/10
263/263 [==============================] - 4s 16ms/step - loss: 0.0699 - accuracy: 0.9783 - val_loss: 0.0822 - val_accuracy: 0.9727
Epoch 5/10
263/263 [==============================] - 4s 16ms/step - loss: 0.0578 - accuracy: 0.9814 - val_loss: 0.0748 - val_accuracy: 0.9750
Epoch 6/10
263/263 [==============================] - 4s 15ms/step - loss: 0.0484 - accuracy: 0.9845 - val_loss: 0.0731 - val_accuracy: 0.9770
Epoch 7/10
263/263 [==============================] - 4s 15ms/step - loss: 0.0416 - accuracy: 0.9870 - val_loss: 0.0708 - val_accuracy: 0.9787
Epoch 8/10

In [18]:
predictions = model.predict(test_ds)

   79/28000 [..............................] - ETA: 35s   

2023-07-13 11:10:02.028208: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [28000,28,28,1]
	 [[{{node Placeholder/_0}}]]


28000/28000 [==============================] - 40s 1ms/step


In [19]:
numerical_predictions = np.argmax(predictions, axis=1)

In [20]:
output_df = pd.read_csv("../submissions/sample_submission.csv")
output_df["Label"] = numerical_predictions

In [21]:
output_df.to_csv("../submissions/lenet_copy.csv", index=False)